In [0]:
import pandas as pd
df1 = pd.read_csv('/dbfs/FileStore/shared_uploads/duvan.robles@metrodebogota.gov.co/adquisicion/identificacion_predios.csv')
df2 = pd.read_csv('/dbfs/FileStore/shared_uploads/duvan.robles@metrodebogota.gov.co/adquisicion/avaluos_comerciales.csv')
# df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/duvan.robles@metrodebogota.gov.co/adquisicion/identificacion_predios.csv")
# df2 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/duvan.robles@metrodebogota.gov.co/adquisicion/avaluos_comerciales.csv")

In [0]:
df1.head()

In [0]:
df2.columns

In [0]:
import pandas as pd
df3 = pd.merge(
    df1, df2,
    how='inner',
    left_on=['IdentificacionPredioId'],
    right_on=['IdentificacionPredioId']).rename(
        columns={'observaciones_x':'observaciones_predio',
                 'observaciones_y':'observaciones_avaluo',
                 })

In [0]:
adquisicion = df3[['chip', 'destino', 'n_estacion', 'valor_comercial', 'daño_emergente', 'lucro_cesante','valor_total']]

In [0]:
veinte23 = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("dbfs:/FileStore/tables/Catastro/2023_octubre/SOL0285553_23.csv")
veinte23 = veinte23.toPandas()

In [0]:
cat2023 = veinte23[['CHIP', 'AREA_TERRENO','AREA_CONSTRUIDA']]

In [0]:
consulta = pd.merge(
    adquisicion, cat2023,
    how='inner',
    left_on=['chip'],
    right_on=['CHIP'])


In [0]:
consulta['valor_comercial'] =  consulta['valor_comercial'].apply(lambda x: x.replace('$','').replace(' ', '').replace('.','').replace(",",'.')).astype(float)
consulta['daño_emergente'] =  consulta['daño_emergente'].apply(lambda x: x.replace('$','').replace(' ', '').replace('.','').replace(",",'.')).astype(float)
#consulta['lucro_cesante'] =  consulta['lucro_cesante'].apply(lambda x: x.replace('$','').replace(' ', '').replace('.','').replace(",",'.').replace('-','').replace('','0')).astype(float)

In [0]:
consulta[(consulta['valor_comercial']>250000000)&(consulta['valor_comercial']<330000000)].sort_values(by=['lucro_cesante'], ascending=False)

In [0]:
consulta['AREA_CONSTRUIDA'] = consulta.AREA_CONSTRUIDA.fillna(0)
consulta['AREA_CONSTRUIDA'] = consulta['AREA_CONSTRUIDA'].astype(str).apply(lambda p: p.replace(",",'.')).astype(float)
consulta['AREA_TERRENO'] = consulta['AREA_TERRENO'].astype(str).apply(lambda p: p.replace(",",'.')).astype(float)
consulta['AT_VM2'] =  consulta['valor_comercial'] / consulta['AREA_TERRENO']
consulta['AC_VM2'] =  consulta['valor_comercial'] / consulta['AREA_CONSTRUIDA']

In [0]:
import numpy as np
consulta = consulta.replace([np.inf, -np.inf], np.nan).fillna(0)

In [0]:
resultado = consulta.groupby(by = ['n_estacion']).agg({'AT_VM2': 'mean', 'AC_VM2': 'mean'}).reset_index()

In [0]:
spark.createDataFrame(resultado).display()